In [1]:
import pandas as pd
import numpy as np

# Load Data

In [2]:
movies = pd.read_csv('ml-latest-small/movies.csv')

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [4]:
tags = pd.read_csv('ml-latest-small/tags.csv')

In [5]:
links = pd.read_csv('ml-latest-small/links.csv')

In [6]:
genome_tags = pd.read_csv('ml-latest-small/genome-tags.csv')

FileNotFoundError: File b'ml-latest-small/genome-tags.csv' does not exist

In [ ]:
genome_scores = pd.read_csv('ml-latest-small/genome-scores.csv')

# Manipulate Data

Extract the year from the movie title and store it in a seperate column (not all movies have the year so those will have a value of 0)

In [ ]:
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)', expand=False).astype('float64').fillna(0).astype('int64')

In [ ]:
movies.dtypes

In [ ]:
movies.shape

In [ ]:
movies.head()

retrieve all the genres available in the movies table

In [ ]:
genres = set(movies['genres'].str.cat(sep='|').split('|'))

In [ ]:
genres

In [ ]:
ratings.dtypes

In [ ]:
ratings.head()

# Data Aggregation 

In [ ]:
movie_data = pd.merge(ratings, movies, on='movieId')

In [ ]:
movie_data.head()

In [ ]:
rating_mean = movie_data.groupby('movieId')['rating'].mean().sort_values(ascending=False).reset_index().rename(columns={'rating':'ratingAvg'})

In [ ]:
rating_mean.head()

In [ ]:
rating_count = movie_data.groupby('movieId')['rating'].count().sort_values(ascending=False).reset_index().rename(columns={'rating':'count'})

In [ ]:
rating_count_mean = pd.merge(rating_count, rating_mean, on='movieId')

In [ ]:
rating_count_mean.head()

In [ ]:
movie_data.loc[movie_data['movieId'] == 1]

# Movie Recommendation System

This bit is all from https://stackabuse.com/creating-a-simple-recommender-system-in-python-using-pandas/

In [ ]:
ratings_mean_count = pd.DataFrame(movie_data.groupby('title')['rating'].mean()) 

In [ ]:
ratings_mean_count.head()

In [ ]:
ratings_mean_count['rating_counts'] = pd.DataFrame(movie_data.groupby('title')['rating'].count())  

In [ ]:
ratings_mean_count.head()

Reset the index as the grouping mixes them up

In [ ]:
ratings_mean_count = ratings_mean_count.reset_index()

In [ ]:
ratings_mean_count.head()

In [ ]:
import matplotlib.pyplot as plt  
import seaborn as sns  
sns.set_style('dark')  
%matplotlib inline

plt.figure(figsize=(8,6))  
plt.rcParams['patch.force_edgecolor'] = True  
ratings_mean_count['rating_counts'].hist(bins=50)  

In [ ]:
plt.figure(figsize=(8,6))  
plt.rcParams['patch.force_edgecolor'] = True  
ratings_mean_count['rating'].hist(bins=50) 

In [ ]:
plt.figure(figsize=(8,6))  
plt.rcParams['patch.force_edgecolor'] = True  
sns.jointplot(x='rating', y='rating_counts', data=ratings_mean_count, alpha=0.4)  

To create the matrix of movie titles and corresponding user ratings

In [ ]:
user_movie_rating = movie_data.to_sparse().pivot_table(index='userId', columns='title', values='rating')  

In [ ]:
user_movie_rating.head()  

In [ ]:
forrest_gump_ratings = user_movie_rating['Forrest Gump (1994)']

In [ ]:
forrest_gump_ratings.head() 

In [ ]:
movies_like_forest_gump = user_movie_rating.corrwith(forrest_gump_ratings)

corr_forrest_gump = pd.DataFrame(movies_like_forest_gump, columns=['Correlation'])  
corr_forrest_gump.dropna(inplace=True)  
corr_forrest_gump.head()  

In [ ]:
corr_forrest_gump.sort_values('Correlation', ascending=False).head(10) 

In [ ]:
corr_forrest_gump = corr_forrest_gump.join(ratings_mean_count['rating_counts'])  
corr_forrest_gump.head() 